# Deepcamp: Codelab 4

**In this tutorial we will cover**:

- Large Language Models (ever heard of ChatGPT 👀 ?)
- LangChain: a tool to allow interaction between LLMs


**Author**:
- Alessio Devoto (alessio.devoto@uniroma1.it)


**Duration**: 50 mins 


**Warning**: Make sure you have an OpenAI token.

In [ ]:
%%capture
!pip install langchain
!pip install openai
!pip install chromadb

In [ ]:
# usual general imports 

import os

# LangChain

Langchain makes interaction with Large Language Models easy and intuitive.

The whole library is comprised of the following building blocks:

- models: a model represents an llm or llm-related service offered by an API endpoint
- prompts: a prompt is what we feed as input to the model
- indexes: an index is a set of rules to split, embed and store documents so that language models can best interact with them
- chains: a sequence of modular components (models, prompts, other chains ...)  combined in a particular way 

The library also offers other modules that we are not interested in today: Agents and Memory. Please refer to the [LangChain official](https://docs.langchain.com/docs/) guide for those.



## Models

Let us start by exploring a simple Langchain Model.
- Model allows us to interact with a number of LLMs providers with a uniform interface. 
- Check the list of supported LLMs [here](https://python.langchain.com/en/latest/modules/models/llms/integrations.html)

We are going to use OpenAI's API, which exposes a lot of [versions](https://platform.openai.com/docs/models/gpt-4) of ChatGPT. 

To do so, you must have an OpenAI account and generate an [access key](https://platform.openai.com/account/api-keys).  

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(
    model_name="text-davinci-002",    # you can pick other models but this is the fastest one                     
    temperature=0 # for OpenAi, see options here: https://platform.openai.com/docs/api-reference/completions/create
    ) 


In [ ]:
# let's ask something to the language model

llm_result = llm("What should I do to learn about Deep Learning and AI ?")
print(llm_result)



There is no one-size-fits-all answer to this question, as the best way to learn about deep learning and AI will vary depending on your level of expertise and experience. However, some suggestions for how to learn about deep learning and AI include attending conferences and workshops, reading books and articles on the topic, and taking online courses.


- llm chain vs llm-index related chain

## Prompts

A prompt is a nice way to format the models input before it is actually fed to the model. 

This can be useful in case we want to keep part of the prompt hidden from the user but still provide an optimal response.

Let us ask a simple question to the LLM. 

In [ ]:
question =  "There are 16 balls. Half of the balls are golf balls. Half of the golf balls are blue balls. How many blue golf balls are there?"

In [ ]:
llm_results = llm(question)
print(llm_results)



There are 8 blue golf balls.


Ok, looks like it got it wrong... 🤓 Can we provide a better question which can help the model give a [better answer](https://arxiv.org/pdf/2205.11916.pdf) ?

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# question will be replaced by user's prompt
template = """
{question}
Let's think about this step by step
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)

final_prompt = prompt.format(riddle=question)

print (f"Final Prompt: {final_prompt}")

print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
There are 16  balls. Half of the balls are golf balls. Half of the golf balls are blue balls. How many blue golf balls are there?
Let's think about this step by step

-----------
LLM Output: There are 16 balls
There are 8 golf balls
There are 4 blue golf balls


## Chains

Chains allow us to chain 😃 modules and prompts in order to attain a task specific goal. 
There are several kinds of chains. We can identify two main categories:

- LLM Chains: interaction of Model, Prompt and OutputParsers
- Index-Related Chains: LLM chains to deal with Documents and External "Memory"

In addition, we have a number of chains that allow you to interact in different fashions with all the modules in the library or even your own local machine processes (See later).

### LLM Chain

A simple LLM Chain accepts two main input arguments:
1. The LLM `Model` to be used
2. The `Prompt` template to be used (if any)

You can also add an output parser at the end of the chain to edit the chain's output.

In [ ]:
from langchain import LLMChain

In [ ]:
# Exercise: Create an LLMChain with a prompt template that takes three input arguments and run with differnt inputs. 
# Follow this template

template = """Write a {adjective} poem about {subject} in {language}."""
prompt = 
llm_chain = LLMChain(
    prompt = ..., 
    llm = ...,
    verbose=True)

results = llm_chain.run(
    adjective=, 
    subject=,
    language=
    )
print(results)


In [ ]:
# Solution
template = """Write a {adjective} poem about {subject} in {language}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject", "language"])
llm_chain = LLMChain(
    prompt=prompt, 
    llm=llm, 
    verbose=True)

results = llm_chain.run(adjective="sad", subject="ducks", language="slang english")
print(results)



> Entering new LLMChain chain...
Prompt after formatting:
Write a sad poem about ducks in slang english.

> Finished chain.


Ducks is some real sad birds
They just waddle around all day
And every time they quack
It just sound like they're sayin' "fuck"

Life ain't easy for no ducks
They got webbed feet and feathers
And they always look so cold
They just waddle around in the water
And every time they try to fly
They just end up fallin' down

Ducks is some real sad birds
And I can't help but feel bad
For everything they go through
They just try to live their lives
The best way they know how
And I hope one day they'll find
A place where they can be happy


In [ ]:
# Just cause it's fun

results = llm_chain.run(adjective="sad", subject="ducks", language="italian")
print(results)



> Entering new LLMChain chain...
Prompt after formatting:
Write a sad poem about ducks in italian.

> Finished chain.


I piccoli anatroccoli

nascono nel fango

e si nutrono di insetti

fino a quando non sono pronti

per volare via

lontano dal loro nido.

Ma prima di allora

i cacciatori vengono

e uccidono i piccoli

anatroccoli

per farne il prezioso

piumaggio.

Ogni anno

i piccoli anatroccoli

muoiono

senza poter mai

volare via.


### Exercise 🏋: Bash Chain

Allocate an LLM Bash chain which chains the LM with a bash process on your local machine. Then ask the llm to create a bash script which echoes `DeepCamp is so cool!` inside the shell.


Follow the steps in the template.

In [ ]:
from langchain.chains import LLMBashChain
from langchain.llms import OpenAI

# 1. Create the language model
llm = 

# 2. Create the chain
bash_chain = LLMBashChain(llm=)

# 3. Create prompt and run the chain
# You should be very clear about what you want the llm to do, as we are using an old version of GPT and it could misunderstand you 



In [ ]:
#@title Peek Solution 👀

from langchain.chains import LLMBashChain
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-002", temperature=0)

text = "Please write a bash script that prints 'DeepCamp is so cool!' to the console."

bash_chain = LLMBashChain(llm=llm, verbose=True)

bash_chain.run(text) # this is reallt being executed in a bash process on your local machine! 



> Entering new LLMBashChain chain...
Please write a bash script that prints 'DeepCamp is so cool!' to the console.

```bash
echo "DeepCamp is so cool!"
```['```bash', 'echo "DeepCamp is so cool!"', '```']

Answer: DeepCamp is so cool!

> Finished chain.


'DeepCamp is so cool!\n'

Now ask the chain to create a directory named `labs_deepcamp`.

In [ ]:
# your code here


In [ ]:
#@title Peek Solution 👀

text = "Please write a bash script that creates a directory named 'labs_deepers'."

bash_chain = LLMBashChain(llm=llm, verbose=True)

bash_chain.run(text)



> Entering new LLMBashChain chain...
Please write a bash script that creates a directory named 'labs_deepers'.

```bash
mkdir labs_deepers
```['```bash', 'mkdir labs_deepers', '```']

Answer: 
> Finished chain.


''

## Indexes 

The concept of Index in LangChain is quite broad. Indexes refer to ways to structure documents so that LLMs can best interact with them.

A naive way of combining LLMs and documents would be injecting the content of a document (assuming it is a text document) into the LLM as a simple prompt. This might work for small docs, as the may possibly fit the LLM [context](https://www.theatlantic.com/technology/archive/2023/03/gpt-4-has-memory-context-window/673426/). When we deal with larger or multiple documents though, this is not feasible. 

We have four classes composing the Index module:

- **Document Loaders**: responsible for loading documents from various sources.

- **Text Splitters**: responsible for splitting text into smaller chunks.

- **VectorStores**: Databases of stored documents

- **Retrievers**: nterface for fetching relevant documents to combine with language models.

[document loader](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html)

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../state_of_the_union.txt', encoding='utf8')

documents = loader.load()



[text splitters](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

[vector stores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

In [ ]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=OpenAIEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
)